In [20]:
import numpy as np
import pandas as pd
import evaluate
import torch
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoTokenizer, pipeline
from sklearn.model_selection import train_test_split

In [21]:
df = pd.read_csv("WELFake_Dataset.csv").drop(columns=["Unnamed: 0"], axis=1)
df['text'] = df['title'].astype(str) + " " + df['text']
df = df.drop(columns=["title"], axis=1)
df

,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,nan Did they post their votes for Hillary alre...,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0
4,SATAN 2: Russia unvelis an image of its terrif...,1
...,...,...
72129,Russians steal research on Trump in hack of U....,0
72130,WATCH: Giuliani Demands That Democrats Apolog...,1
72131,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,Trump tussle gives unpopular Mexican leader mu...,0


In [22]:
df["text"] = df["text"].map(str)

In [23]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [24]:
def preprocess_function(examples):
    return tokenizer(examples, truncation=True, padding=True)

In [25]:
X = df["text"].map(preprocess_function)
y = df["label"]

In [26]:
train_encodings, test_encodings, train_labels, test_labels = train_test_split(X, y, test_size=0.25)

In [27]:
def Text_dict(X):
    X_ = {"input_ids" : [], "attention_mask" : []}
    for i in X:
        X_["input_ids"].append(i["input_ids"])
        X_["attention_mask"].append(i["attention_mask"])

    return X_

In [28]:
def Labels_list(y):
    y_ = []
    for i in y:
        y_.append(i)
    return y_

In [29]:
train_encodings_ = Text_dict(train_encodings)
train_encodings = train_encodings_
test_encodings_ = Text_dict(test_encodings)
test_encodings = test_encodings_

In [30]:
train_labels_ = Labels_list(train_labels)
train_labels = train_labels_
test_labels_ = Labels_list(test_labels)
test_labels = test_labels_

In [31]:
class Fraud_EMail_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.encodings["input_ids"][idx])
        target_ids = torch.tensor(self.labels[idx])
        return {"input_ids": input_ids, "labels": target_ids}

    def __len__(self):
        return len(self.labels)

train = Fraud_EMail_Dataset(encodings=train_encodings, labels=train_labels)
test = Fraud_EMail_Dataset(encodings=test_encodings, labels=test_labels)

In [32]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [33]:
accuracy = evaluate.load("accuracy")

In [34]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [35]:
id2label = {0: "RELIABLE", 1: "UNRELIABLE"}
label2id = {"RELIABLE": 0, "UNRELIABLE": 1}

In [36]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [38]:
training_args = TrainingArguments(
    output_dir="fake_news_detector",
    learning_rate=2e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/21640 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.2225, 'learning_rate': 1.9537892791127542e-05, 'epoch': 0.05}
{'loss': 0.0882, 'learning_rate': 1.9075785582255083e-05, 'epoch': 0.09}
{'loss': 0.0792, 'learning_rate': 1.8613678373382627e-05, 'epoch': 0.14}
{'loss': 0.0736, 'learning_rate': 1.8151571164510168e-05, 'epoch': 0.18}
{'loss': 0.063, 'learning_rate': 1.768946395563771e-05, 'epoch': 0.23}
{'loss': 0.0595, 'learning_rate': 1.7227356746765253e-05, 'epoch': 0.28}
{'loss': 0.0381, 'learning_rate': 1.676524953789279e-05, 'epoch': 0.32}
{'loss': 0.0482, 'learning_rate': 1.6303142329020334e-05, 'epoch': 0.37}
{'loss': 0.0505, 'learning_rate': 1.5841035120147875e-05, 'epoch': 0.42}
{'loss': 0.0474, 'learning_rate': 1.5378927911275416e-05, 'epoch': 0.46}
{'loss': 0.0556, 'learning_rate': 1.4916820702402958e-05, 'epoch': 0.51}
{'loss': 0.0334, 'learning_rate': 1.44547134935305e-05, 'epoch': 0.55}
{'loss': 0.05, 'learning_rate': 1.3992606284658041e-05, 'epoch': 0.6}
{'loss': 0.0385, 'learning_rate': 1.3530499075785584e-05, '

  0%|          | 0/3607 [00:00<?, ?it/s]

{'eval_loss': 0.02586720883846283, 'eval_accuracy': 0.9950648774536985, 'eval_runtime': 355.3204, 'eval_samples_per_second': 50.754, 'eval_steps_per_second': 10.151, 'epoch': 1.0}
{'loss': 0.0427, 'learning_rate': 9.833641404805916e-06, 'epoch': 1.02}
{'loss': 0.0091, 'learning_rate': 9.371534195933457e-06, 'epoch': 1.06}
{'loss': 0.0139, 'learning_rate': 8.909426987060998e-06, 'epoch': 1.11}
{'loss': 0.0142, 'learning_rate': 8.44731977818854e-06, 'epoch': 1.16}
{'loss': 0.0291, 'learning_rate': 7.985212569316083e-06, 'epoch': 1.2}
{'loss': 0.0142, 'learning_rate': 7.523105360443623e-06, 'epoch': 1.25}
{'loss': 0.0115, 'learning_rate': 7.060998151571166e-06, 'epoch': 1.29}
{'loss': 0.0131, 'learning_rate': 6.5988909426987065e-06, 'epoch': 1.34}
{'loss': 0.0234, 'learning_rate': 6.136783733826248e-06, 'epoch': 1.39}
{'loss': 0.0093, 'learning_rate': 5.67467652495379e-06, 'epoch': 1.43}
{'loss': 0.0092, 'learning_rate': 5.212569316081332e-06, 'epoch': 1.48}
{'loss': 0.0103, 'learning_rat

  0%|          | 0/3607 [00:00<?, ?it/s]

{'eval_loss': 0.021916883066296577, 'eval_accuracy': 0.9965620494621271, 'eval_runtime': 349.5045, 'eval_samples_per_second': 51.599, 'eval_steps_per_second': 10.32, 'epoch': 2.0}
{'train_runtime': 7626.5149, 'train_samples_per_second': 14.187, 'train_steps_per_second': 2.837, 'train_loss': 0.034863851746878206, 'epoch': 2.0}


TrainOutput(global_step=21640, training_loss=0.034863851746878206, metrics={'train_runtime': 7626.5149, 'train_samples_per_second': 14.187, 'train_steps_per_second': 2.837, 'train_loss': 0.034863851746878206, 'epoch': 2.0})

In [39]:
trainer.save_model()